This notebook is a demonstration of how TRS can be used to retrieve workflows and its files.

In [1]:
!pip install git+https://github.com/ga4gh/fasp-clients

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/ga4gh/fasp-clients to /tmp/pip-req-build-8a01s1h2
  Running command git clone --filter=blob:none --quiet https://github.com/ga4gh/fasp-clients /tmp/pip-req-build-8a01s1h2
  Resolved https://github.com/ga4gh/fasp-clients to commit bdc02d354841bd485fccf99bcd91537c5904ef2f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


We can view the available TRS services by using the `GA4GHRegistryClient`.

In [2]:
from fasp.loc import GA4GHRegistryClient
cl = GA4GHRegistryClient('https://registry.ga4gh.org/v1')
cl.getRegisteredServices('org.ga4gh:trs')

Searching the GA4GH registry for org.ga4gh:trs services


[{'id': 'org.dockstore.dockstoreapi',
  'name': 'Dockstore',
  'type': {'group': 'org.ga4gh', 'artifact': 'trs', 'version': '2.0.0'},
  'organization': {'id': 'org.dockstore',
   'name': 'Dockstore',
   'url': 'https://dockstore.org'},
  'version': '1.8.1',
  'url': 'https://dockstore.org/api/api/ga4gh/v2/',
  'description': 'Dockstore TRS implementation',
  'documentationUrl': 'https://dockstore.org/api/static/swagger-ui/index.html'},
 {'id': 'pro.biocontainers.biocontainersapi',
  'name': 'Biocontainers',
  'type': {'group': 'org.ga4gh', 'artifact': 'trs', 'version': '2.0.0'},
  'organization': {'id': 'pro.biocontainers',
   'name': 'Biocontainers',
   'url': 'https://biocontainers.pro'},
  'version': '1.0.0',
  'url': 'https://api.biocontainers.pro/api/ga4gh/v2',
  'description': 'Biocontainers TRS implementation',
  'documentationUrl': 'https://biocontainers-edu.biocontainers.pro/en/latest/'}]

Dockstore is one of the TRS services listed, so we will use it in this notebook. Another TRS service that we will use is WorkflowHub, however, it is not listed in the registry.

Below are some TRS constants that will be used in this notebook.

In [3]:
DOCKSTORE_TRS_URL = 'https://dockstore.org/api/ga4gh/trs/v2/'
# The following Dockstore TRS ID and version ID are for this workflow: https://dockstore.org/workflows/github.com/dockstore/dockstore-tool-bamstats/wdl:1.25-9?tab=info
DOCKSTORE_EXAMPLE_TRS_ID = '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl'
DOCKSTORE_EXAMPLE_TRS_VERSION_ID = '1.25-9'

WORKFLOW_HUB_TRS_URL = 'https://workflowhub.eu/ga4gh/trs/v2/'

First, we will use the WorkflowHub TRS service to demonstrate how to get information about the TRS service, what tool types are available, and how to list tools.

In [4]:
trsUrl = WORKFLOW_HUB_TRS_URL

Below are helper functions that will execute the TRS requests and print the responses in a user friendly way.

In [5]:
import requests, json

def getTrsRequest(urlSuffix):
  fullTrsUrl = trsUrl + urlSuffix
  print("Performing GET request with URL " + fullTrsUrl)
  return requests.get(fullTrsUrl)

def getTrsRequestAsJson(urlSuffix):
  return getTrsRequest(urlSuffix).json()

def prettyPrintJson(jsonObject):
    print(json.dumps(jsonObject, indent=4))

def urlEncode(stringToEncode):
  return requests.utils.quote(stringToEncode, safe='') # Set safe to nothing because it considers slashes safe by default

Get information about the WorkflowHub TRS service.

In [6]:
prettyPrintJson(getTrsRequestAsJson('service-info'))

Performing GET request with URL https://workflowhub.eu/ga4gh/trs/v2/service-info
{
    "contactUrl": "mailto:WORKFLOWHUB-ADMIN@listserv.manchester.ac.uk",
    "description": "TRS API endpoint for WorkflowHub",
    "documentationUrl": "https://editor.swagger.io/?url=https://raw.githubusercontent.com/ga4gh/tool-registry-service-schemas/release/v2.0.1/openapi/openapi.yaml",
    "environment": "production",
    "id": "eu.workflowhub",
    "name": "WorkflowHub",
    "organization": {
        "name": "WorkflowHub",
        "url": "https://about.workflowhub.eu/"
    },
    "type": {
        "artifact": "trs",
        "group": "org.ga4gh",
        "version": "2.0.1"
    },
    "version": "1.14.1"
}


These are the types of tools that can be retrieved using WorkflowHub's TRS service. WorkflowHub only has workflows.

In [7]:
prettyPrintJson(getTrsRequestAsJson('toolClasses'))

Performing GET request with URL https://workflowhub.eu/ga4gh/trs/v2/toolClasses
[
    {
        "id": "1",
        "name": "Workflow",
        "description": "A computational workflow"
    }
]


You can list all the tools available using the `tools` endpoint. This endpoint also allows you to filter tools and page through the results using the `offset` parameter.

Let's list 3 workflows using this TRS endpoint. For this example, we set the `limit` query parameter to `3` because we only want to see 3 results and the `offset` to 0 because we want to view the first page of results.

In [8]:
listToolsUrl = 'tools?toolClass={toolClass}&limit={limit}&offset={offset}'

prettyPrintJson(getTrsRequestAsJson(listToolsUrl.format(toolClass = 'Workflow', limit = '3', offset = 0)))

Performing GET request with URL https://workflowhub.eu/ga4gh/trs/v2/tools?toolClass=Workflow&limit=3&offset=0
[
    {
        "id": "2",
        "url": "https://workflowhub.eu/workflows/2",
        "name": "Genomics - Read pre-processing",
        "description": "Preprocessing of raw SARS-CoV-2 reads. More info can be found at https://covid19.galaxyproject.org/genomics/",
        "organization": "GalaxyProject SARS-CoV-2",
        "toolclass": {
            "id": "1",
            "name": "Workflow",
            "description": "A computational workflow"
        },
        "versions": [
            {
                "id": "1",
                "url": "https://workflowhub.eu/workflows/2?version=1",
                "name": "Version 1",
                "author": [],
                "descriptor_type": [
                    "GALAXY"
                ]
            }
        ]
    },
    {
        "id": "3",
        "url": "https://workflowhub.eu/workflows/3",
        "name": "Virus genome assemb

Next, we will use the Dockstore TRS service to demonstrate how you can retrieve information about a specific tool, its versions, and its files. We will be using retrieving information about [this Dockstore workflow](https://dockstore.org/workflows/github.com/dockstore/dockstore-tool-bamstats/wdl:1.25-9?tab=info) using the TRS API.

In [9]:
# Choose one of DOCKSTORE_TRS_URL or WORKFLOW_HUB_TRS_URL here
trsUrl = DOCKSTORE_TRS_URL
exampleTrsId = DOCKSTORE_EXAMPLE_TRS_ID
exampleVersionId = DOCKSTORE_EXAMPLE_TRS_VERSION_ID

print("Using TRS URL " + trsUrl)
print("Using example TRS ID '" + exampleTrsId + "'")
print("Using example TRS version ID '" + exampleVersionId + "'")

Using TRS URL https://dockstore.org/api/ga4gh/trs/v2/
Using example TRS ID '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl'
Using example TRS version ID '1.25-9'


You can retrieve a specific tool by specifying the ID using the `tools/{id}` endpoint.

In [10]:
print("Getting tool with ID '" + exampleTrsId + "'\n")
prettyPrintJson(getTrsRequestAsJson('tools/{id}'.format(id = urlEncode(exampleTrsId))))

Getting tool with ID '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl'

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl
{
    "aliases": [],
    "checker_url": "",
    "description": "# dockstore-tool-bamstats\n\nA repo for the `Dockerfile` to create a Docker image for the BAMStats command. Also contains the\n`Dockstore.yml` which is used by the [Dockstore](https://www.dockstore.org) to register\nthis container and describe how to call BAMStats for the community.\n\n## Validation \n\nThis tool has been validated as a CWL draft-3 and v1.0 CommandLineTool. \n\nVersions that we tested with are documented iin https://raw.githubusercontent.com/dockstore/dockstore/1.13.0-beta.3/dockstore-webservice/src/main/resources/requirements/1.13.0/requirements3.txt\n\n\n## Building Manually\n\nNormally you would let GitHub actions build this.  But, if you need to build manually (for example due 

The following examples will use this endpoint prefix that includes the TRS ID so that we can perform various actions on it, such as retrieve its versions, get a specific version and its files.

Let's start with listing the versions using the `tools/{id}/versions` endpoint.

In [11]:
# List versions
prettyPrintJson(getTrsRequestAsJson('tools/{id}/versions'.format(id = urlEncode(exampleTrsId))))

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions
[
    {
        "author": [
            "Andrew Duncan"
        ],
        "containerfile": false,
        "descriptor_type": [
            "WDL"
        ],
        "descriptor_type_version": {
            "WDL": [
                "1.0"
            ]
        },
        "id": "#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl:develop",
        "images": [],
        "included_apps": [],
        "is_production": false,
        "meta_version": "2022-08-17 18:03:25.0",
        "name": "develop",
        "signed": false,
        "url": "https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/develop",
        "verified": false,
        "verified_source": []
    },
    {
        "author": [
            "Andrew Duncan"
        ],
        "containerfile": false,
     

Let's get a specific version using the `tools/{id}/versions/{versionId}` endpoint.

In [12]:
print("Getting version '{versionId}' for tool '{id}'\n".format(versionId = exampleVersionId, id = exampleTrsId))

version = getTrsRequestAsJson('tools/{id}/versions/{versionId}'.format(id = urlEncode(exampleTrsId), versionId = exampleVersionId))
prettyPrintJson(version)

Getting version '1.25-9' for tool '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl'

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/1.25-9
{
    "author": [
        "Andrew Duncan"
    ],
    "containerfile": false,
    "descriptor_type": [
        "WDL"
    ],
    "descriptor_type_version": {
        "WDL": [
            "1.0"
        ]
    },
    "id": "#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl:1.25-9",
    "images": [
        {
            "checksum": [
                {
                    "checksum": "171725ff643f6a2e12468a780e7969c4ccc368afbcf8e0195c3b12b3ab48c602",
                    "type": "sha-256"
                }
            ],
            "image_name": "ghcr.io/dockstore/dockstore-tool-bamstats:1.25-8",
            "image_type": "Docker",
            "registry_host": "ghcr.io",
            "size": 457240691,
            "updated": null
 

Now that we have the version, we can get a list of files that the version has using the `tools/{id}/versions/{versionId}/{type}/files` endpoint. In order to do this, we need to provide the type of the descriptor, which was provided in the `descriptor_type` key in the response body of the previous API call when we retrieved the version.

In [13]:
# Get the descriptor type
descriptorType = version['descriptor_type'][0]

# Get a list of files that the version has
print("Getting a list of files for tool with ID '{id}', version '{versionId}', and type '{type}'\n".format(id = exampleTrsId, versionId = exampleVersionId, type = descriptorType))
files = getTrsRequestAsJson('tools/{id}/versions/{versionId}/{type}/files'.format(id = urlEncode(exampleTrsId), versionId = exampleVersionId, type = descriptorType))
prettyPrintJson(files)

Getting a list of files for tool with ID '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl', version '1.25-9', and type 'WDL'

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/1.25-9/WDL/files
[
    {
        "checksum": {
            "checksum": "888a220be757db31d32e4ad0281b3eab716cbb12bb6a3eaff5b52b5796a1f7e1",
            "type": "sha-256"
        },
        "file_type": "OTHER",
        "path": ".dockstore.yml"
    },
    {
        "checksum": {
            "checksum": "c3e6cfba2851d3bcc88396c59351c091e4755ef2246aa9313b1c4deb3214807e",
            "type": "sha-256"
        },
        "file_type": "PRIMARY_DESCRIPTOR",
        "path": "bamstats.wdl"
    },
    {
        "checksum": {
            "checksum": "c3bf71fa0060bb2f095cba44a235b91c5f0d1c414235b7d4e0eb0964261cd6fa",
            "type": "sha-256"
        },
        "file_type": "TEST_FILE",
        "path": "test

With this endpoint, you can also download these files as a zip file by specifying `format=zip`. We can see that the zip file has three files - the same three files that were listed in the response body above.

In [14]:
# Download the files as a zip
print("Downloading a zip file containing files for tool with ID '{id}', version '{versionId}', and type '{type}'\n".format(id = exampleTrsId, versionId = exampleVersionId, type = descriptorType))
fileZipResponse = getTrsRequest('tools/{id}/versions/{versionId}/{type}/files?format=zip'.format(id = urlEncode(exampleTrsId), versionId = exampleVersionId, type = descriptorType))
with open('files.zip', 'wb') as f:
    f.write(fileZipResponse.content)

# View the zip file
!unzip -l files.zip



Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/1.25-9/WDL/files?format=zip
Archive:  files.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
      694  2024-02-08 16:05   .dockstore.yml
      511  2024-02-08 16:05   bamstats.wdl
       90  2024-02-08 16:05   test.wdl.json
---------                     -------
     1295                     3 files


We can get the version's descriptor file using the `tools/{id}/versions/{versionId}/{type}/descriptor` endpoint.

In [15]:
# Get the descriptor
print("Getting the descriptor for tool with ID '{id}', version '{versionId}', and type '{type}'\n".format(id = exampleTrsId, versionId = exampleVersionId, type = descriptorType))
prettyPrintJson(getTrsRequestAsJson('tools/{id}/versions/{versionId}/{type}/descriptor'.format(id = urlEncode(exampleTrsId), versionId = exampleVersionId, type = descriptorType)))

Getting the descriptor for tool with ID '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl', version '1.25-9', and type 'WDL'

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/1.25-9/WDL/descriptor
{
    "checksum": [
        {
            "checksum": "c3e6cfba2851d3bcc88396c59351c091e4755ef2246aa9313b1c4deb3214807e",
            "type": "sha-256"
        }
    ],
    "content": "version 1.0\ntask bamstats {\n    input {\n        File bam_input\n        Int mem_gb\n    }\n\n\n\tcommand {\n\t\tbash /usr/local/bin/bamstats ${mem_gb} ${bam_input}\n\t}\n\n\toutput {\n\t\tFile bamstats_report = \"bamstats_report.zip\"\n\t}\n\n\truntime {\n\t\tdocker: \"ghcr.io/dockstore/dockstore-tool-bamstats:1.25-8\"\n\t\tmemory: mem_gb + \"GB\"\n\t}\n\n\tmeta {\n\t\tauthor: \"Andrew Duncan\"\n\t}\n}\n\nworkflow bamstatsWorkflow {\n    input {\n        File bam_input\n        Int mem_gb\n    

If you want the raw content of the descriptor file, you can prefix the `type` with `PLAIN_`. For example, if the type is `WDL`, use `PLAIN_WDL`.

In [16]:
plainType = 'PLAIN_' + descriptorType
print("Getting the plain descriptor for tool with ID '{id}', version '{versionId}', and type '{type}'\n".format(id = exampleTrsId, versionId = exampleVersionId, type = plainType))
plainDescriptorResponse = getTrsRequest('tools/{id}/versions/{versionId}/{type}/descriptor'.format(id = urlEncode(exampleTrsId), versionId = exampleVersionId, type = plainType))

# Write content to a file
descriptorFileName = 'descriptor.' + descriptorType.lower()
with open(descriptorFileName, 'wb') as f:
    f.write(plainDescriptorResponse.content)

print("\nSaved descriptor to file '{descriptorFileName}'. Displaying file content:\n".format(descriptorFileName = descriptorFileName))
!cat {descriptorFileName}

Getting the plain descriptor for tool with ID '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl', version '1.25-9', and type 'PLAIN_WDL'

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/1.25-9/PLAIN_WDL/descriptor

Saved descriptor to file 'descriptor.wdl'. Displaying file content:

version 1.0
task bamstats {
    input {
        File bam_input
        Int mem_gb
    }


	command {
		bash /usr/local/bin/bamstats ${mem_gb} ${bam_input}
	}

	output {
		File bamstats_report = "bamstats_report.zip"
	}

	runtime {
		docker: "ghcr.io/dockstore/dockstore-tool-bamstats:1.25-8"
		memory: mem_gb + "GB"
	}

	meta {
		author: "Andrew Duncan"
	}
}

workflow bamstatsWorkflow {
    input {
        File bam_input
        Int mem_gb
    }
	call bamstats { input: bam_input=bam_input, mem_gb=mem_gb }
}


Using the descriptor file as an anchor, we can retrieve other files using relative paths with the `tools/{id}/versions/{versionId}/{type}/descriptor/{relativePath}` endpoint. Let's get a file using its path. We will use the file path of the first file returned from the `tools/{id}/versions/{versionId}/{type}/files` endpoint

Note: you may also prefix the type with `PLAIN_` for this endpoint if you want the raw file content.

In [17]:
filePath = files[0]['path'] 
print("Getting file with relative path '{filePath}' for tool with ID '{id}', version '{versionId}', and type '{type}'\n".format(filePath = filePath, id = exampleTrsId, versionId = exampleVersionId, type = descriptorType))
prettyPrintJson(getTrsRequestAsJson('tools/{id}/versions/{versionId}/{type}/descriptor/{relativePath}'.format(id = urlEncode(exampleTrsId), versionId = exampleVersionId, type = descriptorType, relativePath = filePath)))

Getting file with relative path '.dockstore.yml' for tool with ID '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl', version '1.25-9', and type 'WDL'

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/1.25-9/WDL/descriptor/.dockstore.yml
{
    "checksum": [
        {
            "checksum": "888a220be757db31d32e4ad0281b3eab716cbb12bb6a3eaff5b52b5796a1f7e1",
            "type": "sha-256"
        }
    ],
    "content": "version: 1.2\ntools:\n  - subclass: CWL\n    primaryDescriptorPath: /bamstats.cwl\n    testParameterFiles:\n      - /test.json\n    name: bamstats_cwl\n    authors:\n      - orcid: 0000-0002-6130-1021\n  - subclass: CWL\n    primaryDescriptorPath: /bamstats_sort.cwl\n    testParameterFiles:\n      - /test.json\n    name: bamstats_sort_cwl\n    authors:\n      - orcid: 0000-0002-6130-1021\nworkflows:\n  - subclass: WDL\n    primaryDescriptorPath: /bamstats.w

We can get a version's test files using the `tools/{id}/versions/{versionId}/{type}/tests` endpoint.

In [18]:
print("Getting test files for tool with ID '{id}', version '{versionId}', and type '{type}'\n".format(filePath = filePath, id = exampleTrsId, versionId = exampleVersionId, type = descriptorType))

prettyPrintJson(getTrsRequestAsJson('tools/{id}/versions/{versionId}/{type}/tests'.format(id = urlEncode(exampleTrsId), versionId = exampleVersionId, type = descriptorType)))

Getting test files for tool with ID '#workflow/github.com/dockstore/dockstore-tool-bamstats/wdl', version '1.25-9', and type 'WDL'

Performing GET request with URL https://dockstore.org/api/ga4gh/trs/v2/tools/%23workflow%2Fgithub.com%2Fdockstore%2Fdockstore-tool-bamstats%2Fwdl/versions/1.25-9/WDL/tests
[
    {
        "checksum": [
            {
                "checksum": "c3bf71fa0060bb2f095cba44a235b91c5f0d1c414235b7d4e0eb0964261cd6fa",
                "type": "sha-256"
            }
        ],
        "content": "{\n  \"bamstatsWorkflow.bam_input\": \"rna.SRR948778.bam\",\n  \"bamstatsWorkflow.mem_gb\": \"4\"\n}\n",
        "image_type": {},
        "url": "https://raw.githubusercontent.com/dockstore/dockstore-tool-bamstats/1.25-9/test.wdl.json"
    }
]
